https://velog.io/@jochedda/NLP-%EC%9E%90%EC%97%B0%EC%96%B4-%EC%A0%84%EC%B2%98%EB%A6%AC

In [2]:
# 한국어 문장 분리기KSS(Korean Sentence Splitter)
!pip install kss

In [3]:
# https://github.com/lovit/soynlp
# 데이터에 존재하는 단어를 찾거나, 문장을 단어열로 분해, 혹은 품사 판별을 할 수 있는 비지도학습 접근법을 지향
!pip install soynlp

In [4]:
import os
import shutil
import pandas as pd
import re
from kss import split_sentences
import emoji
from soynlp.normalizer import repeat_normalize

In [6]:
# os.getcwd()
%cd  C:\Users\vision21\Documents\PCIT\data
%pwd


C:\Users\vision21\Documents\PCIT\data


'C:\\Users\\vision21\\Documents\\PCIT\\data'

In [7]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: AE35-3D1B

 C:\Users\vision21\Documents\PCIT\data 디렉터리

2023-09-15  오후 02:24    <DIR>          .
2023-09-15  오후 02:24    <DIR>          ..
2023-09-15  오전 10:22             9,829 PCIT_09152023.csv
               1개 파일               9,829 바이트
               2개 디렉터리  916,131,774,464 바이트 남음


In [120]:
# emojis = ''.join(emoji.UNICODE_EMOJI.keys())
# # emojis = ''.join(emoji.UNICODE_EMOJI.keys()) # UNICODE_EMOJI에 에러가 있어 수정함.
# pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
# url_pattern = re.compile(
#     r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

# def clean(x):
#     x = pattern.sub(' ', x)
#     x = url_pattern.sub('', x)
#     x = x.strip()
#     x = repeat_normalize(x, num_repeats=2)
#     return x

In [8]:
rawdata = pd.read_csv('./PCIT_09152023.csv')
rawdata.head(50)

,label,parent
0,중립적 말,오늘 우리 마당에 귀여운 토끼가 있었어
1,부정적 말,날 귀찮게 하고있어
2,반영,포테이토 헤드를 위한 모자이구나
3,명령,그림에 말을 그리렴
4,행동 묘사,우리는 색칠을 하고 있어
5,질문,음 알겠어
6,부정적 말,색칠한 그림이 엉망이구나
7,중립적 말,테이블 위에 서있는 건 용납되지 않아
8,반영,집에 가고 싶구나
9,부정적 말,아니야


In [17]:
type(len(rawdata.index))

int

In [10]:
for i in range(len(rawdata.index)):
    rawdata['parent'][i] = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", rawdata['parent'][i])
    rawdata['parent'][i] = re.sub(r'[키키]', "", rawdata['parent'][i])
    rawdata['parent'][i] = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', "", rawdata['parent'][i])
    rawdata['parent'][i] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', "", rawdata['parent'][i])
rawdata.head(50)

,label,parent
0,중립적 말,오늘 우리 마당에 귀여운 토끼가 있었어
1,부정적 말,날 귀찮게 하고있어
2,반영,포테이토 헤드를 위한 모자이구나
3,명령,그림에 말을 그리렴
4,행동 묘사,우리는 색칠을 하고 있어
5,질문,음 알겠어
6,부정적 말,색칠한 그림이 엉망이구나
7,중립적 말,테이블 위에 서있는 건 용납되지 않아
8,반영,집에 가고 싶구나
9,부정적 말,아니야


In [11]:
rawdata.to_csv('preprocess_PCIT_09152023.csv')

In [122]:
# input = '\ㅜ'
# clean(input)

# re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", input)

''

In [136]:
# n_dir = ['kakao1','kakao2', 'kakao3','kakao4', 'facebook', 'instagram', 'band', 'nateon']
# # for dir_name  in n_dir:
# path_07 = 'C:/Users/vision21/Downloads/020.주제별 텍스트 일상 대화 데이터/01.데이터/1.Training/원천데이터/' + n_dir[7]
# file_list = os.listdir(path_07)

with open('PCIT_09152023.csv', 'w') as fw:
    f = open(PCIT_09152023.csv, encoding='UTF8')
    for line_b in f.readlines():
        try:
            split_line = re.split(':', line_b)
            line = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", split_line[1])
            line = re.sub(r'[키키]', '', line) # 하하, 와 

#                 print(line)
            for split_line_again in split_sentences(line):
#                     print(split_line_again)
                fw.write(split_line_again +'\n')
#                 fw.write('\n')
        except IndexError:
            pass

#         print(file_name)

